<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Generative%20Models/Llama%202%20Hugging%20Face%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram

### Setup

In [1]:
!pip install transformers accelerate

In [ ]:
!huggingface-cli login

### Imports

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

### Initializing the tokenizer

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
sequences = pipeline(
    'Select all employees. Convert this statement to SQL query',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Select all employees. Convert this statement to SQL query.

Answer: Sure! Here is the SQL query to convert the given statement to:

\begin{code}
SELECT * FROM employees;
\end{code}

Explanation:

* `SELECT` is the keyword used to select rows from a table.
* `*` is a wildcard character that selects all columns from the table.
* `EMPLOYEES` is the name of the table you want to select rows from.

So, the SQL query `SELECT * FROM EMPLOYEES;` will select all rows from the `EMPLOYEES` table.
